In [44]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pandas as pd
import pymongo
from config import mongoConn

In [45]:
# URL of page to be scraped
query = 'sabrina'
base_url = 'https://reelgood.com'
url = f'{base_url}/search?q={query}'
url

'https://reelgood.com/search?q=sabrina'

In [46]:
# Retrieve page with the requests module
response = requests.get(url)

In [47]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')
link = soup.select('.e1qyeclq5 a')[0]['href']
show_url = base_url + link
show_url

'https://reelgood.com/movie/sabrina-2018'

In [48]:
show_response = requests.get(show_url)
show_soup = BeautifulSoup(show_response.text, 'html.parser')
#print(show_soup.prettify())

In [49]:
#services = show_soup.select('.e126mwsw1 span[class*=hou11]')
services = show_soup.select('.e126mwsw1 span[class*=hou113]')
services

[<span class="css-3g9tm3 e1udhou113">Netflix</span>]

In [50]:
for service in services:
    print(service.text)

Netflix


In [51]:
dic_list = {
    'query': query,
    'services': [s.text for s in set(services)],
    'title': query,
    'description': query
}

dic_list

{'query': 'sabrina',
 'services': ['Netflix'],
 'title': 'sabrina',
 'description': 'sabrina'}

In [52]:
#conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(mongoConn)

db = client.shows_db
collection = db.items


In [53]:
collection.insert_one(dic_list)

In [54]:
results = collection.find({}, {'_id': False})
coll_df = pd.DataFrame(results)
coll_json = coll_df.to_json(orient='records')

coll_json

'[{"query":"dexter","services":["fuboTV","Showtime","Prime Video","Rent or Buy"],"title":"dexter","description":"dexter"},{"query":"cobra kai","services":["YouTube Premium","Netflix","Rent or Buy"],"title":"cobra kai","description":"cobra kai"},{"query":"sabrina","services":["Netflix"],"title":"sabrina","description":"sabrina"}]'

In [55]:
results = collection.find({}, {'_id': False})
for result in results:
    print(result)


{'query': 'dexter', 'services': ['fuboTV', 'Showtime', 'Prime Video', 'Rent or Buy'], 'title': 'dexter', 'description': 'dexter'}
{'query': 'cobra kai', 'services': ['YouTube Premium', 'Netflix', 'Rent or Buy'], 'title': 'cobra kai', 'description': 'cobra kai'}
{'query': 'sabrina', 'services': ['Netflix'], 'title': 'sabrina', 'description': 'sabrina'}
